In [52]:
import sqlite3
import pandas as pd

# Kết nối đến database
conn = sqlite3.connect("database.db")
cursor = conn.cursor()

# Xóa bảng nếu tồn tại
cursor.execute("DROP TABLE IF EXISTS student;")
cursor.execute("DROP TABLE IF EXISTS course;")

conn.commit()  # Lưu thay đổi
conn.close()  # Đóng kết nối

print("✅ Đã xóa hai bảng student và course!")


✅ Đã xóa hai bảng student và course!


In [53]:
import sqlite3
import pandas as pd

with sqlite3.connect("database.db") as conn:
    cursor = conn.cursor()


In [54]:
# Tạo bảng course
cursor.execute('''
CREATE TABLE IF NOT EXISTS course (
    id INTEGER PRIMARY KEY,
    course_name TEXT NOT NULL
);
''')

# Tạo bảng student
cursor.execute('''
CREATE TABLE IF NOT EXISTS student (
    student_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    class TEXT,
    course_id INTEGER,
    score REAL,
    FOREIGN KEY (course_id) REFERENCES course(id)
);
''')

conn.commit()  # Lưu thay đổi


In [55]:
# Thêm dữ liệu vào course
cursor.executemany('''
INSERT INTO course (id, course_name) VALUES (?, ?)''', [
    (12, 'Giai tich'),
    (34, 'Thong ke'),
    (26, 'Tin hoc')
])

# Thêm dữ liệu vào student
cursor.executemany('''
INSERT INTO student (student_id, name, class, course_id, score) 
VALUES (?, ?, ?, ?, ?)''', [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
])

conn.commit()
print("Dữ liệu đã được thêm thành công!")


Dữ liệu đã được thêm thành công!


In [56]:
# (1) Tích Descartes
print(" 1. Tích Descartes:")
df = pd.read_sql_query("SELECT * FROM student, course;", conn)
print(df, "\n")

# (2) INNER JOIN
print(" 2. INNER JOIN:")
df = pd.read_sql_query('''
SELECT student.*, course.course_name 
FROM student 
INNER JOIN course 
ON student.course_id = course.id;
''', conn)
print(df, "\n")

# (3) LEFT JOIN
print(" 3. LEFT JOIN:")
df = pd.read_sql_query('''
SELECT student.*, course.course_name 
FROM student 
LEFT JOIN course 
ON student.course_id = course.id;
''', conn)
print(df, "\n")

# (4) RIGHT JOIN (SQLite không hỗ trợ RIGHT JOIN nên ta dùng LEFT JOIN đảo ngược)
print(" 4. RIGHT JOIN (mô phỏng bằng LEFT JOIN):")
df = pd.read_sql_query('''
SELECT student.*, course.course_name 
FROM course 
LEFT JOIN student 
ON student.course_id = course.id;
''', conn)
print(df, "\n")

# Chạy FULL OUTER JOIN
query = '''
SELECT student.*, course.course_name 
FROM student 
LEFT JOIN course 
ON student.course_id = course.id

UNION

SELECT student.*, course.course_name 
FROM student 
LEFT JOIN course 
ON student.course_id = course.id;
'''

df = pd.read_sql_query(query, conn)
print(df)

conn.close()

 1. Tích Descartes:
    student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
3            2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich
4            2       Tran Thi Lan   Kinh Te       34.0    9.2  26     Tin hoc
5            2       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
6            3       Pham Van Nam  Toan Tin        NaN    7.9  12   Giai tich
7            3       Pham Van Nam  Toan Tin        NaN    7.9  26     Tin hoc
8            3       Pham Van Nam  Toan Tin        NaN    7.9  34    Thong ke
9            4     Le Thanh Huyen  Toan Tin       20.0    7.2  12   Giai tich
10           4     Le Thanh Huyen  Toan Tin       20.0    7.2  26     Tin hoc
11           4     Le Thanh Huyen  Toan Tin 

In [65]:
#  Dùng `with` để đảm bảo kết nối không bị đóng trước khi thực hiện truy vấn
with sqlite3.connect("database.db") as conn:
    cursor = conn.cursor()

#  Cập nhật course_id còn thiếu
cursor.execute("UPDATE student SET course_id = 26 WHERE course_id IS NULL;")
conn.commit()

#  Xóa bản ghi không hợp lệ
cursor.execute("DELETE FROM student WHERE course_id NOT IN (SELECT id FROM course);")
conn.commit()

#  Tổng số sinh viên & Điểm trung bình theo lớp
df = pd.read_sql_query('''
SELECT class, COUNT(*) AS total_students, AVG(score) AS avg_score 
FROM student 
GROUP BY class;
''', conn)
print(" Tổng số sinh viên & Điểm trung bình theo lớp:\n", df)

#  Tổng số sinh viên & Điểm trung bình theo môn học
df = pd.read_sql_query('''
SELECT course.course_name, COUNT(student.student_id) AS total_students, AVG(student.score) AS avg_score 
FROM student 
JOIN course ON student.course_id = course.id 
GROUP BY course.course_name;
''', conn)
print(" Tổng số sinh viên & Điểm trung bình theo môn học:\n", df)

#  Phân loại thi đua theo môn học
df = pd.read_sql_query('''
SELECT course.course_name, 
       CASE 
           WHEN AVG(score) >= 9.0 THEN 'Xuất sắc'
           WHEN AVG(score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
           ELSE 'Kém'
       END AS ranking
FROM student 
JOIN course ON student.course_id = course.id 
GROUP BY course.course_name;
''', conn)
print(" Phân loại thi đua theo môn học:\n", df)

conn.close()

 Tổng số sinh viên & Điểm trung bình theo lớp:
       class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               1   7.900000
 Tổng số sinh viên & Điểm trung bình theo môn học:
   course_name  total_students  avg_score
0   Giai tich               1   6.700000
1    Thong ke               2   9.200000
2     Tin hoc               3   7.366667
 Phân loại thi đua theo môn học:
   course_name   ranking
0   Giai tich       Tốt
1    Thong ke  Xuất sắc
2     Tin hoc       Tốt


In [64]:
#  Dùng `with` để đảm bảo kết nối không bị đóng trước khi thực hiện truy vấn
with sqlite3.connect("database.db") as conn:
    cursor = conn.cursor()

    #  Cập nhật course_id còn thiếu (Gán mặc định 26 - 'Tin hoc')
    cursor.execute("UPDATE student SET course_id = 26 WHERE course_id IS NULL;")
    conn.commit()

    #  Xóa các bản ghi không hợp lệ (course_id không tồn tại)
    cursor.execute("DELETE FROM student WHERE course_id NOT IN (SELECT id FROM course);")
    conn.commit()

    #  Tổng số sinh viên & Điểm trung bình theo lớp
    df = pd.read_sql_query('''
    SELECT class, COUNT(*) AS total_students, AVG(score) AS avg_score 
    FROM student 
    GROUP BY class;
    ''', conn)
    print(" Tổng số sinh viên & Điểm trung bình theo lớp:\n", df)

    #  Tổng số sinh viên & Điểm trung bình theo môn học
    df = pd.read_sql_query('''
    SELECT course.course_name, COUNT(student.student_id) AS total_students, AVG(student.score) AS avg_score 
    FROM student 
    JOIN course ON student.course_id = course.id 
    GROUP BY course.course_name;
    ''', conn)
    print(" Tổng số sinh viên & Điểm trung bình theo môn học:\n", df)

    #  Phân loại thi đua theo môn học
    df = pd.read_sql_query('''
    SELECT course.course_name, 
           CASE 
               WHEN AVG(score) >= 9.0 THEN 'Xuất sắc'
               WHEN AVG(score) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
               ELSE 'Kém'
           END AS ranking
    FROM student 
    JOIN course ON student.course_id = course.id 
    GROUP BY course.course_name;
    ''', conn)
    print(" Phân loại thi đua theo môn học:\n", df)

    #  Xếp hạng sinh viên theo điểm số
    df = pd.read_sql_query('''
    SELECT student_id, name, score, 
           RANK() OVER (ORDER BY score DESC) AS rank_position
    FROM student;
    ''', conn)
    print(" Xếp hạng sinh viên theo điểm số chung:\n", df)

    #  Xếp hạng sinh viên theo lớp
    df = pd.read_sql_query('''
    SELECT student_id, name, class, score, 
           RANK() OVER (PARTITION BY class ORDER BY score DESC) AS rank_in_class
    FROM student;
    ''', conn)
    print(" Xếp hạng sinh viên theo lớp:\n", df)

    #  Xếp hạng sinh viên theo môn học
    df = pd.read_sql_query('''
    SELECT student_id, name, course_id, score, 
           RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS rank_in_course
    FROM student;
    ''', conn)
    print(" Xếp hạng sinh viên theo môn học:\n", df)

    #  Top 3 sinh viên cao nhất & thấp nhất theo điểm số chung
    df = pd.read_sql_query('''
    WITH RankedStudents AS (
        SELECT student_id, name, score, 
               RANK() OVER (ORDER BY score DESC) AS rank_position
        FROM student
    )
    SELECT * FROM RankedStudents WHERE rank_position <= 3;
    ''', conn)
    print(" Top 3 sinh viên điểm cao nhất:\n", df)

    df = pd.read_sql_query('''
    WITH RankedStudents AS (
        SELECT student_id, name, score, 
               RANK() OVER (ORDER BY score ASC) AS rank_position
        FROM student
    )
    SELECT * FROM RankedStudents WHERE rank_position <= 3;
    ''', conn)
    print(" Top 3 sinh viên điểm thấp nhất:\n", df)

print(" Hoàn thành tất cả truy vấn mà không bị lỗi database đóng!")


 Tổng số sinh viên & Điểm trung bình theo lớp:
       class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               1   7.900000
 Tổng số sinh viên & Điểm trung bình theo môn học:
   course_name  total_students  avg_score
0   Giai tich               1   6.700000
1    Thong ke               2   9.200000
2     Tin hoc               3   7.366667
 Phân loại thi đua theo môn học:
   course_name   ranking
0   Giai tich       Tốt
1    Thong ke  Xuất sắc
2     Tin hoc       Tốt
 Xếp hạng sinh viên theo điểm số chung:
    student_id               name  score  rank_position
0           2       Tran Thi Lan    9.2              1
1           7      Bui Tien Dung    9.2              1
2           3       Pham Van Nam    7.9              3
3           9     Duong Huu Phuc    7.2              4
4          10       Cao Thi Hanh    7.0              5
5           1  Nguyen Minh Hoang    6.7              6
 Xếp hạng sinh viên theo